# <center> Классификация изображений</center>

(Данные и исходные файлы смотри в: **materials\6. 02.10 - Image classify**)

In [107]:
import numpy as np
import pandas as pd
from os import walk
from PIL import Image, ImageFilter, ImageOps    
from sklearn.model_selection import train_test_split

from sklearn.neighbors import KNeighborsClassifier

## Загрузка обработанных изображений

In [108]:
img_dir = './cropped/'
_, _, up = next(walk(img_dir + 'up'))
_, _, down = next(walk(img_dir + 'down'))
_, _, mov = next(walk(img_dir + 'mov'))
up = [img_dir + 'up/' + filename for filename in up]
down = [img_dir + 'down/' + filename for filename in down]
mov = [img_dir + 'mov/' + filename for filename in mov]

In [109]:
filenames = up + down + mov
y_full = ['up']*len(up) + ['down']*len(down) + ['mov']*len(mov) 
n_max = len(filenames)
part_coeff = 0.1
int(n_max * part_coeff)

5082

In [110]:
sample_idx = sorted(np.random.choice(np.arange(len(filenames)), int(n_max * part_coeff), replace=False))
# id отобранных изображений

In [111]:
# пути до отобранных изображений:
filenames_sample = np.array(filenames)[sample_idx]
# целевые признаки отобранных изображений
y = np.array(y_full)[sample_idx]

In [112]:
X = []
for img_path in filenames_sample:
    img = Image.open(img_path)
    X.append(np.hstack(np.array(img)))
X = np.array(X)

In [113]:
# выборка данных, где картинки уже преобразованы в массивы (не в матрицы)
X.shape

(5082, 2100)

## Выделение обучающей и контрольной выборок

In [114]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

<div class="alert alert-info">

<h3> Задание.</h3>
<p></p>

 <ol>
  <li>Решите задачу классификации изображений двумя методами: методом опорных векторов (sklearn.svm.SVC) и методом ближайших соседей (sklearn.neighbors.KNeighborsClassifier). Оцените качество классификации с помощью матрицы ошибок (sklearn.metrics.confusion_matrix).</li>
 <p></p>

      
  <li>Определите оптимальный параметр n_neighbors у KNeighborsClassifier с помощью sklearn.model_selection.GridSearchCV. Метрикой качества выберите f1.</li>
<p></p>
  

 </ol> 
</div>

**Задание 1**

Для спокойствия перекодируем целевые признаки в числа:

In [115]:
np.unique(y_train)

array(['down', 'mov', 'up'], dtype='<U4')

In [116]:
yc_train = np.where(y_train=='down', 0, np.where(y_train=='mov', 1, 2))
np.unique(yc_train)

array([0, 1, 2])

In [117]:
yc_test = np.where(y_test=='down', 0, np.where(y_test=='mov', 1, 2))
np.unique(yc_test)

array([0, 1, 2])

**Метод опорных векторов**

In [118]:
from sklearn.svm import SVC

Модель:

In [119]:
model_SVM = SVC(decision_function_shape='ovo')

In [120]:
model_SVM.fit(X_train, yc_train)

SVC(decision_function_shape='ovo')

Оценка качества:

In [121]:
from sklearn.metrics import confusion_matrix

In [122]:
predcit_SVM = model_SVM.predict(X_test)

In [123]:
m_SVM = confusion_matrix(yc_test, predcit_SVM)
m_SVM

array([[389,   0,   0],
       [ 54, 109,  43],
       [ 18,   1, 403]], dtype=int64)

In [124]:
t = np.array([[1,2,3],[4,5,6],[7,8,9]])
t

array([[1, 2, 3],
       [4, 5, 6],
       [7, 8, 9]])

Расчитаем $F$-меру:

In [127]:
def F_score(m):
    d = m.diagonal()
    f1 = 2 * d/m.sum(axis=1) * d/m.sum(axis=0) / (d/m.sum(axis=1) + d/m.sum(axis=0))
    # f1, F-мера, F-взешн. многокласс.
    return f1, f1.mean(), m.sum(axis=0)@f1 / m.sum()

In [128]:
f1_svm, F_svm, Fm_svm = F_score(m_SVM)
print('f1:', f1_svm)
print('F:', F_svm)
print('Fвз.мн.:', Fm_svm)

f1: [0.91529412 0.68987342 0.92857143]
F: 0.8445796546466687
Fвз.мн.: 0.896735025887432


**Метод ближайших соседей**

Модель:

In [129]:
from sklearn.neighbors import KNeighborsClassifier

In [130]:
model_KNN = KNeighborsClassifier()

In [131]:
model_KNN.fit(X_train, yc_train)

KNeighborsClassifier()

Оценка качества:

In [132]:
predcit_KNN = model_KNN.predict(X_test)

In [133]:
m_KNN = confusion_matrix(yc_test, predcit_KNN)
m_KNN

array([[366,   7,  16],
       [ 22, 166,  18],
       [  4,   5, 413]], dtype=int64)

In [134]:
f1_knn, F_knn, Fm_knn = F_score(m_KNN)
print('f1:', f1_knn)
print('F:', F_knn)
print('Fвз.мн.:', Fm_knn)

f1: [0.93725992 0.86458333 0.95051784]
F: 0.9174536977008452
Fвз.мн.: 0.9303669578913656


Можно судить о незначительной разнице качества за вычетом более равномерного распределния у метода ближайших соседей, а также о более высокой $F$-мере у KNN.

**Задание 2**

In [81]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score, make_scorer

In [74]:
model_GS_KNN = KNeighborsClassifier()

In [75]:
params = {
    'n_neighbors': [5, 7, 10, 15, 30, 50]
}

In [90]:
f1_scorer = make_scorer(f1_score, y_true=yc_train, average='weighted')

In [91]:
gs = GridSearchCV(model_GS_KNN, params, scoring=f1_scorer)

In [92]:
gs.fit(X_train, yc_train)

GridSearchCV(estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': [5, 7, 10, 15, 30, 50]},
             scoring=make_scorer(f1_score, y_true=[0 2 0 ... 0 0 2], average=weighted))

Лучший параметр:

In [99]:
sorted(zip(gs.cv_results_['mean_test_score'], gs.cv_results_['params']),
       key=lambda x: x[0], reverse=True)[0]

(0.9296539724712639, {'n_neighbors': 5})